In [21]:
import sys

sys.path.append("../../src")

In [22]:
import numpy as np
import gaussian_overlap
import importlib
import h5py
import pandas as pd
from scipy.linalg import eigh
import matplotlib.pyplot as plt
import copy

from scipy.optimize import minimize

importlib.reload(gaussian_overlap)
from gaussian_overlap import *
from sto_ng_matrix import *

In [23]:
df = pd.read_csv("../make_base/sto_3g.csv")
df

,exps_1,coefs_1s,exps_2,coefs_2s,coefs_2p
0,0.109987,0.445576,0.074759,0.694673,0.387976
1,0.407154,0.535362,0.229124,0.405616,0.608867
2,2.246014,0.153482,0.979606,-0.100240,0.158557


In [27]:
zeta = 1.685
param1s = {
    "center": np.array([0, 0, 0]),
    "lmn": (0, 0, 0),
    "exps": np.array(df["exps_1"]) * zeta**2,
    "coefs": np.array(df["coefs_1s"]),
}
param2s = {
    "center": np.array([0, 0, 0]),
    "lmn": (0, 0, 0),
    "exps": np.array(df["exps_2"]) * zeta**2,
    "coefs": np.array(df["coefs_2s"]),
}
param2px = {
    "center": np.array([0, 0, 0]),
    "lmn": (1, 0, 0),
    "exps": np.array(df["exps_2"]) * zeta**2,
    "coefs": np.array(df["coefs_2p"]),
}
param2py = {
    "center": np.array([0, 0, 0]),
    "lmn": (0, 1, 0),
    "exps": np.array(df["exps_2"]) * zeta**2,
    "coefs": np.array(df["coefs_2p"]),
}
param2pz = {
    "center": np.array([0, 0, 0]),
    "lmn": (0, 0, 1),
    "exps": np.array(df["exps_2"]) * zeta**2,
    "coefs": np.array(df["coefs_2p"]),
}
base1s = BasisFunction(**param1s)
base2s = BasisFunction(**param2s)
base2px = BasisFunction(**param2px)
base2py = BasisFunction(**param2py)
base2pz = BasisFunction(**param2pz)
# basis = [base1s, base2s, base2px, base2py, base2pz]
basis = [base1s]
Tmat = get_Tmat(basis)
Vmat = -2 * get_Vmat(basis, np.array([0, 0, 0]))
ERImat = get_ERImat(basis)
Smat = get_Smat(basis)

In [32]:
hmat = Tmat + Vmat
c_up = [[1]]
c_dn = [[1]]
new_energy = 0
for i in range(1000):
    Jmat_up = sum(
        np.einsum("ijkl,k, l->ij", ERImat, c_up[i], c_up[i]) for i in range(len(c_up))
    ) + sum(
        np.einsum("ijkl,k, l->ij", ERImat, c_dn[i], c_dn[i]) for i in range(len(c_dn))
    )
    Kmat_up = sum(
        np.einsum("ijkl,i, l->jk", ERImat, c_up[i], c_up[i]) for i in range(len(c_up))
    )
    Fmat_up = hmat + Jmat_up - Kmat_up
    Jmat_dn = sum(
        np.einsum("ijkl,i, l->jk", ERImat, c_up[i], c_up[i]) for i in range(len(c_up))
    ) + sum(
        np.einsum("ijkl,k, l->ij", ERImat, c_dn[i], c_dn[i]) for i in range(len(c_dn))
    )
    Kmat_dn = sum(
        np.einsum("ijkl,i, l->jk", ERImat, c_dn[i], c_dn[i]) for i in range(len(c_dn))
    )
    Fmat_dn = hmat + Jmat_dn - Kmat_dn
    val_up, vec_up = eigh(Fmat_up, Smat)
    val_dn, vec_dn = eigh(Fmat_dn, Smat)
    c_up = vec_up[:, :1].T
    c_dn = vec_dn[:, :1].T
    old_energy = new_energy
    new_energy = (
        val_up[0]
        + val_dn[0]
        - c_up @ (Jmat_up - Kmat_up) @ c_up.T * 0.5
        - c_dn @ (Jmat_dn - Kmat_dn) @ c_dn.T * 0.5
    )


print(c_up)
print(c_dn)
new_energy

[[1.]]
[[1.]]


array([[-2.80825178]])